### Test the usage of random forest
Training a Random forest \
This is a new random forest trained from scratch \
Sep 4, 2020

In [ ]:
import numpy as np
import glob
import os
import pandas as pd
import pickle
import time

In [35]:
import matplotlib.pyplot as plt

In [36]:
%matplotlib widget

In [37]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_curve

In [ ]:
## Old labels
# fname='/global/cfs/cdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/autoscan_features.3.csv'

In [40]:
## New labels
fname='/global/cfs/cdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/relabeled_autoscan_features.csv'
df_full=pd.read_csv(fname,sep=',',comment='#')

### Convert BAND to integers
bm = {'g':0, 'r':1, 'i':2, 'z':3}
df_full['BAND']=np.array([bm[i] for i in df_full.BAND.values])
df_full.head()

,ID,OBJECT_TYPE,AMP,A_IMAGE,A_REF,BAND,B_IMAGE,B_REF,CCDID,COLMEDS,...,N3SIG5,N3SIG5SHIFT,NN_DIST_RENORM,NUMNEGRN,SCALE,SNR,SPREADERR_MODEL,SPREAD_MODEL,old_OBJECT_TYPE,check_relabel
0,10742010,0,0.808323,1.508,2.650069,2,0.950,1.899501,10,0.112077,...,0,-9,0.674934,22,2.024122,7.722346,0.004629,-0.003718,0,False
1,8828139,0,0.975141,1.430,NaN,2,1.319,NaN,10,0.072256,...,0,5,NaN,21,1.379283,25.582918,0.002420,-0.000578,0,False
2,8229889,0,0.877440,0.505,NaN,3,0.494,NaN,57,0.106124,...,0,1,NaN,20,0.950069,3.914192,0.008863,-0.020044,0,False
3,11033218,0,0.681179,0.762,3.008957,2,0.449,2.855881,1,0.139071,...,2,-27,1.240574,18,1.749488,5.068382,0.008041,0.000246,0,False
4,8341858,0,0.645906,1.205,1.352224,2,0.982,1.063333,30,0.123302,...,0,4,2.970234,16,1.930613,7.372579,0.005606,0.001771,0,False


In [41]:
df=df_full[:].dropna()
df.shape

y_cols=['OBJECT_TYPE']
## Find columns with Nans
# df.columns[df_full.isna().any().values]
# reject_cols=['A_REF', 'B_REF', 'MAG_REF', 'MAG_REF_ERR', 'NN_DIST_RENORM']
reject_cols=[]
x_cols=[i for i in df.columns if i not in ['OBJECT_TYPE','ID','old_OBJECT_TYPE','check_relabel']+reject_cols]


In [42]:
### Get IDs for test data
results_dir='/global/project/projectdirs/dasrepo/vpa/supernova_cnn/data/results_data/results/'
# fname='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Supernova-cnn/cnn_supernova/4_train_random_forest/train_ids.train'
fname=results_dir+'id_train_4.train'
train_ids=np.loadtxt(fname,dtype=np.int32)[:]
fname=results_dir+'id_test_4.test'
test_ids=np.loadtxt(fname,dtype=np.int32)[:]


# Get train data
# train_size=801000
## IDs not in test ids
df_temp=df[~df_full.ID.isin(test_ids)].dropna()
# df_temp=df[df.ID.isin(train_ids)].dropna()
X_train,y_train=df_temp[x_cols],df_temp[y_cols]

# Get test data . This should be the same set used for testing the CNNs
df_temp=df_full[df_full.ID.isin(test_ids)].dropna()
X_test,y_test=df_temp[x_cols],df_temp[y_cols]
print(X_train.shape,X_test.shape)

<ipython-input-42-a3441038a021>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp=df[~df_full.ID.isin(test_ids)].dropna()


(674512, 38) (168637, 38)


In [43]:
X_train.shape,X_test.shape

((674512, 38), (168637, 38))

In [23]:
# df=df.dropna()

# X=df[x_cols]
# y=df[y_cols]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# print(X_train.shape,X_test.shape)

### Test with Kfold CV

In [24]:
# X_train.reset_index(),y_train.reset_index();

# kf=StratifiedKFold(n_splits=4,shuffle=True)
# model=RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=-1, max_features=6,
#                              max_depth=None, min_samples_split=3, verbose=1)


# fprs,tprs,scores=[],[],[]
# for train_idx,test_idx in kf.split(X_train,y_train):
#     print(train_idx.shape,test_idx.shape)
#     print(np.mean(train_idx),np.mean(test_idx))
#     model.fit(X_train.iloc[train_idx],y_train.iloc[train_idx].OBJECT_TYPE.values)
    
#     y_predict=model.predict_proba(X_train.iloc[test_idx])
#     fpr,tpr,thrs=roc_curve(y_train.iloc[test_idx],y_predict[:,1])
#     fprs.append(fpr)
#     tprs.append(tpr)
    
    

In [17]:
# plt.figure()
# for i in range(len(fprs)):
#     fpr,tpr=fprs[i],tprs[i]
#     x,y=fpr,(1-tpr)
#     plt.plot(x, y,linestyle='',markersize=2,marker='*')
#     # ### Reference points in mdr plot in paper
    
# plt.plot(0.03,0.038,marker='s',markersize=8,color='k')
# plt.plot(0.04,0.024,marker='s',markersize=8,color='k')
# plt.plot(0.05,0.016,marker='s',markersize=8,color='k')

# plt.xlabel('MDR')
# plt.ylabel('FPR')
# plt.xlim(0,0.1)
# plt.ylim(0,0.05)

### Reading feature weights in paper

In [ ]:
# ### Load importance of features 

# fle='features.txt'
# a1=np.loadtxt(fle,delimiter=',',dtype=str)

# weight_dict1={}
# for i in range(a1.shape[0]):
#     key,val=a1[i][0],a1[i][1]
#     weight_dict1[key.upper()]=val

# # [i for i in df_full.columns if i not in weight_dict.keys()],[i for i in weight_dict.keys() if i not in df.columns]
    
# key_maps={'R_APER_PSF':'GFLUX','DIFFSUM':'DIFFSUMRN','MIN_DIST_TO_EDGE':'MIN_DISTANCE_TO_EDGE_IN_NEW','NUMNEG':'NUMNEGRN','BANDNUM':'BAND'}
# # Renaming keys to match those in DataFrame
# weight_dict={}
# for key in weight_dict1.keys():
#     if key not in key_maps:
#         weight_dict[key]=float(weight_dict1[key])
#     else : 
#         weight_dict[key_maps[key]]=float(weight_dict1[key])

# # weight_dict

### Run model

In [44]:
### Train model
t1=time.time()
model=RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=10, max_features=6,
                             max_depth=None, min_samples_split=3, verbose=1)
model.fit(X_train,y_train.OBJECT_TYPE.values)
t2=time.time()
print("Training time",t2-t1)



[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   23.3s


Training time 75.64207053184509


[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:  1.3min finished


In [45]:
### Test model
t1=time.time()
y_predict=model.predict_proba(X_test)
t2=time.time()
print("Time of testing model",t2-t1)
fpr,tpr,thrs=roc_curve(y_test,y_predict[:,1])

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.1s


Time of testing model 0.5207455158233643


[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    0.4s finished


In [46]:
model.score(X_test,y_test.values)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    0.4s finished


0.972639456347065

In [47]:
# fpr2,tpr2=fpr,tpr

### Plot roc curve

In [48]:
def f_roc(fpr,tpr,label):
    x,y=fpr,(1-tpr)
#     plt.figure()
    plt.plot(x, y,label=label,linestyle='',markersize=2,marker='*')
    # ### Reference points in mdr plot in paper
    plt.plot(0.03,0.038,marker='s',markersize=8,color='k')
    plt.plot(0.04,0.024,marker='s',markersize=8,color='k')
    plt.plot(0.05,0.016,marker='s',markersize=8,color='k')

    plt.xlabel('MDR')
    plt.ylabel('FPR')
    plt.xlim(0,0.1)
    plt.ylim(0,0.05)


In [49]:
plt.figure()
f_roc(fpr,tpr,'rf_final')
# f_roc(fpr2,tpr2,'train_800k')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
np.array_equal(X_test.AMP.values,df[df.ID.isin(test_ids)].AMP.values)

True

In [51]:
df_save=pd.DataFrame([])
df_save['ID']=df[df.ID.isin(test_ids)].ID.values
df_save['m_rf']=y_predict[:,1]

In [52]:
df_save.head()

,ID,m_rf
0,11390373,0.00
1,7895795,0.00
2,9346337,0.00
3,11781734,0.06
4,11525250,0.02


In [53]:
# df_save.to_csv('rf_preds_old.csv',index=False)
df_save.to_csv('rf_preds_relabeled.csv',index=False)